null, [] and [null] are equivalent

lets add some more test data

In [1]:
curl -XPOST 'localhost:9200/my_index/posts/_bulk?pretty' -H 'Content-Type: application/json' -d'
{ "index": { "_id": "1"              }}
{ "tags" : ["search"]                }  
{ "index": { "_id": "2"              }}
{ "tags" : ["search", "open_source"] }  
{ "index": { "_id": "3"              }}
{ "other_field" : "some data"        }  
{ "index": { "_id": "4"              }}
{ "tags" : null                      }  
{ "index": { "_id": "5"              }}
{ "tags" : ["search", null]          }
'

{
  "took" : 408,
  "errors" : false,
  "items" : [
    {
      "index" : {
        "_index" : "my_index",
        "_type" : "posts",
        "_id" : "1",
        "_version" : 1,
        "result" : "created",
        "_shards" : {
          "total" : 2,
          "successful" : 2,
          "failed" : 0
        },
        "created" : true,
        "status" : 201
      }
    },
    {
      "index" : {
        "_index" : "my_index",
        "_type" : "posts",
        "_id" : "2",
        "_version" : 1,
        "result" : "created",
        "_shards" : {
          "total" : 2,
          "successful" : 2,
          "failed" : 0
        },
        "created" : true,
        "status" : 201
      }
    },
    {
      "index" : {
        "_index" : "my_index",
        "_type" : "posts",
        "_id" : "3",
        "_version" : 1,
        "result" : "created",
        "_shards" : {
          "total" : 2,
          "successful" : 2,
          "failed" : 0
        },
        "created" : true,
  

find all values where a tag is set using the exists query

SELECT tags
FROM   posts
WHERE  tags IS NOT NULL

In [2]:
curl -XGET 'localhost:9200/my_index/posts/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "query" : {
        "constant_score" : {
            "filter" : {
                "exists" : { "field" : "tags" }
            }
        }
    }
}
'

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 3,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "my_index",
        "_type" : "posts",
        "_id" : "5",
        "_score" : 1.0,
        "_source" : {
          "tags" : [
            "search",
            null
          ]
        }
      },
      {
        "_index" : "my_index",
        "_type" : "posts",
        "_id" : "2",
        "_score" : 1.0,
        "_source" : {
          "tags" : [
            "search",
            "open_source"
          ]
        }
      },
      {
        "_index" : "my_index",
        "_type" : "posts",
        "_id" : "1",
        "_score" : 1.0,
        "_source" : {
          "tags" : [
            "search"
          ]
        }
      }
    ]
  }
}


use the must not exists to find the opposite

SELECT tags
FROM   posts
WHERE  tags IS NULL

In [5]:
curl -XGET 'localhost:9200/my_index/posts/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "query": {
        "constant_score" : { 
            "filter" : {
                "bool": {
                    "must_not": {
                        "exists" : { "field" : "tags" }
                    }
                }
            }
        }
    }
}
'

{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 2,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "my_index",
        "_type" : "posts",
        "_id" : "4",
        "_score" : 1.0,
        "_source" : {
          "tags" : null
        }
      },
      {
        "_index" : "my_index",
        "_type" : "posts",
        "_id" : "3",
        "_score" : 1.0,
        "_source" : {
          "other_field" : "some data"
        }
      }
    ]
  }
}


exists works on nested fields too!

{
   "name" : {
      "first" : "John",
      "last" :  "Smith"
   }
}

is flattened as:

{
   "name.first" : "John",
   "name.last"  : "Smith"
}

and the query:

{
    "exists" : { "field" : "name" }
}

is really executed as:

{
    "bool": {
        "should": [
            { "exists": { "field": "name.first" }},
            { "exists": { "field": "name.last" }}
        ]
    }
}